In [13]:
import pandas as pd
from fbprophet import Prophet
import plotly.express as px
import plotly.io as pio
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt
from datetime import date
import os
from PIL import Image

In [14]:
tech_stocks = ["FB", "AAPL", "AMZN", "TSLA", "GOOG", "NVDA", "NFLX"]
dfs = list()
prop_dfs = list()

In [15]:
for i in tech_stocks:
    if not os.path.isdir("output/" + str(i)):
        os.makedirs("output/" + str(i))
        
    if not os.path.isdir("output/" + str(i) + "/graphs"):
        os.makedirs("output/" + str(i) + "/graphs")
        
    if not os.path.isdir("output/" + str(i) + "/files"):
        os.makedirs("output/" + str(i) + "/files")    
        
        
    yf.pdr_override()
    today_date = date.today()
    dfs.append(pdr.get_data_yahoo(i, start = today_date - dt.timedelta(days = 5 * 365), end = today_date))

In [16]:
def initializer(df, c):
    if 'Date' not in df.columns:
        df = df.reset_index()
        
    f = open("output/" + tech_stocks[c - 1] + "/files/analysis.txt" , "w")
    #f = open("files/file" + str((c % len(tech_stocks)) + 1) + ".txt" , "w")
    f.write("Stock Information for Ticker: " + str(tech_stocks[c-1]) + "\n")
    f.write(df.mean().to_string())
    f.write("\n\n\n")
    f.write(df.describe().to_string())
    f.close()
    
    return df

In [17]:
def visualize(df, c):
    figs = list()
    figs.append(px.area(df, x = "Date", y = "Close"))
    figs.append(px.line(df, x = "Date", y = "Close"))
    figs.append(px.area(df, x = "Date", y = "Volume"))
    figs.append(px.line(df, x = "Date", y = "Volume"))   
    figs.append(px.bar(df, y = "Volume"))
    figs.append(px.box(df, y = "Close"))
    
    for i, j in enumerate(figs):
        j.write_image("output/" + str(tech_stocks[c-1]) + "/graphs/occurred" + str(i+1) + ".png")

In [18]:
def preprocess(df):
    columns = ["Date", "Close"]
    prop_df = pd.DataFrame(df, columns = columns)
    prop_df = prop_df.rename(columns = {"Date": "ds", "Close": "y"})
    return prop_df

In [19]:
def forecaster(prop_df, c):
    m = Prophet()
    m.fit(prop_df)
    
    future = m.make_future_dataframe(periods = 36, freq = "M")
    forecast = m.predict(future)
    f = open("output/" + tech_stocks[c - 1] + "/files/analysis.txt" , "a")
    f.write("\n\n\n")
    f.write(forecast.to_string())
    f.close()
    #pred_visualization(forecast, m, c)

In [20]:
def pred_visualization(fc, m, num):
    figs = list()
    figs.append(px.line(fc, x = "ds", y = "yhat"))
    figs.append(m.plot(fc, xlabel = "ds", ylabel = "y"))
    figs.append(m.plot_components(fc))
    for i, j in enumerate(figs):
        j.write_image("output/" + str(tech_stocks[num-1]) + "/graphs/forecast" + str(i+1) + ".png")

In [21]:
for i in range(len(dfs)):
    dfs[i] = initializer(dfs[i], i+1)
    visualize(dfs[i], i)
    prop_dfs.append(preprocess(dfs[i]))
    forecaster(prop_dfs[i], i+1)

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_63989/1205042290.py:8: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -19.5422
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        3604.4    0.00495324        746.66           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3647.62     0.0532339       1841.21           1           1      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3664.72   0.000752391       273.385           1           1      358   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     349       3667.94   0.000136428       276.681   5.086e-07       0.001      454  LS failed, Hessian reset 
     399       3669.11   0.000439325       161.424      0.8839      0.8839      512   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     447       3670.35   3.86302e-05    

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_63989/1205042290.py:8: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -23.4191
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3618.63     0.0442816       2232.99           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3725.13     0.0184533       463.896           1           1      230   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        3745.1    0.00288411       464.665           1           1      359   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3751.18   0.000548883       76.3452           1           1      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     487       3754.96   0.000136403        341.38   1.911e-07       0.001      619  LS failed, Hessian reset 
     499       3756.56   0.000357893    

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_63989/1205042290.py:8: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Initial log joint probability = -29.081
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       4052.43    0.00402607       694.468           1           1      128   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4134.55    0.00355015       215.639           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       4172.33    0.00327316       1267.84      0.1815      0.1815      354   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       4201.37     0.0283272        3985.7           1           1      468   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       4222.36    0.00472684       117.032           1           1      590   
    Iter      log prob        ||dx||      ||grad||       alpha   

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_63989/1205042290.py:8: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -43.7134
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3649.44    0.00630454       622.272           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3709.92    0.00388751       690.038           1           1      247   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3740.56     0.0217875       1279.04           1           1      360   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3762.11    0.00249495         459.4      0.5643      0.5643      479   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       3770.35    0.00385983       314.229      0.7277      0.7277      599   
    Iter      log prob        ||dx||      ||grad||       alpha  

/var/folders/s1/y8hjc_rd18g3zw5xh5l4_y8r0000gn/T/ipykernel_63989/1205042290.py:8: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.53714
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3341.74      0.762909       4280.55           1           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       3395.38     0.0180832       614.888           1           1      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       3429.15     0.0764658       1318.42           1           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     307       3430.91    0.00010058       143.282   2.671e-07       0.001      400  LS failed, Hessian reset 
     399       3441.79    0.00105877       183.915      0.4172      0.4172      513   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     422       3442.75   6.18691e-05    